# Semidefinite relaxations of discrete problems

Semidefinite programs often play a role in the design of approximation
algorithms for hard discrete problems. This notebook shows a famous example of
this and introduces some useful properties of semidefinite matrices.

## Preamble

In [ ]:
import numpy as np
import networkx as nx
from scipy import spatial
from plotly import offline, graph_objects as go

COORDS_KEY = "coords"
WEIGHT_KEY = "weight"
LAYOUT = dict(
    scene=dict(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False),
    yaxis=dict(scaleanchor="x1"), margin=dict(l=10, r=10, t=10, b=10),
    paper_bgcolor="white", plot_bgcolor="rgba(0,0,0,0)",
)

offline.init_notebook_mode()


## Problem 2: Maximum cut in undirected graphs

### Setting

A *cut* in a weighted undirected graph $I = (V, E)$ with edge weights $w_e \geq
0$ is a partition of the vertex set $V$ into two disjoint subsets $C$ and $V
\setminus C$. In the *minimum* cut problem, one considers in addition two
terminal vertices $s \neq t \in V$ and requires that $s \in C$ and $t \in V
\setminus C$. Here the goal is to choose $C$ such that the total weight of
edges leaving $C$, formally

$$
    w(C) = \sum_{e \in E, \; |e \cap C| = 1} w_e
$$

is minimized. It is well known that such a minimum cut [can be computed
efficiently via linear
programming](https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem).

The *maximum* cut problem, on the other end, is an $\mathsf{NP}$-complete
problem, so no polynomial time algorithm can exist for it unless $\mathsf{P} =
\mathsf{NP}$. Here, we do not consider terminal vertices&mdash;$C \subseteq V$
may be chosen arbitrarily&mdash;and we want to maximize $w(C)$. This problem
pops up, for instance, in [statistical
mechanics](https://en.wikipedia.org/wiki/Ising_model#Connection_to_graph_maximum_cut).

Let's generate a random graph to work with:

In [ ]:
n = 40  # number of vertices

def make_weighted_graph(points):
    """Create a weighted graph from a Delaunay triangulation of 2D points.

    Edge weights are set to the Euclidean distances between the points.
    """
    G = nx.Graph()

    triangulation = spatial.Delaunay(points)

    for path in triangulation.simplices:
        nx.add_path(G, path)

    for v in G.nodes:
        G.nodes[v][COORDS_KEY] = triangulation.points[v, :]

    nx.set_edge_attributes(
        G,
        {
            (u, v): np.linalg.norm(
                G.nodes[u][COORDS_KEY] - G.nodes[v][COORDS_KEY]
            )
            for u, v in G.edges
        },
        WEIGHT_KEY,
    )

    return G

def vert_data(G, select=None):
    """Report vertex coordinates of a graph for plotting with plotly."""
    x_vec, y_vec = [], []

    for u, d in G.nodes(data=True):
        if select is None or select[u]:
            x, y = d[COORDS_KEY]
            x_vec.append(x)
            y_vec.append(y)

    return dict(x=x_vec, y=y_vec)

def edge_data(G, x=None, only_cut=None):
    """Report edge coordinates of a graph for plotting with plotly."""
    x_seq, y_seq = [], []

    for u, v, d in G.edges(data=True):
        if x is None or (x[u]*x[v] > 0) ^ only_cut:
            ux, uy = G.nodes[u][COORDS_KEY]
            vx, vy = G.nodes[v][COORDS_KEY]
            x_seq.extend((ux, vx, None))
            y_seq.extend((uy, vy, None))

    return dict(x=x_seq, y=y_seq)

# Create a graph from random points.
points = np.random.random((n, 2))
G = make_weighted_graph(points)

# Plot the graph.
traces = [
    go.Scatter(
        mode="lines+markers", **edge_data(G),
        line_color="lightgray", marker_color="black"),
]
go.Figure(traces, LAYOUT).show()

In the above graph, the weight of an edge is set to the Euclidean distance
between its endpoints. We are looking to partition the vertices such that the
weight of edges between the partitions is as large as possible.

### The problem

Since we cannot hope to find an efficient algorithm that always produces an
optimum solution to the maximum cut problem, we will have to do with an
approximation. An approximation algorithm has ratio $\rho$ if for every problem
instance $I$ (here $I = G$), the value of the solution returned by the
algorithm, $\operatorname{ALG}(I)$, is always within a factor of $\rho$ within
the optimum value $\operatorname{OPT}(I)$, formally $\rho = \sup_I
\frac{\operatorname{ALG}(I)}{\operatorname{OPT}(I)} \geq 1$ for minimization
problems and $\rho = \inf_I \frac{\operatorname{ALG}(I)}{\operatorname{OPT}(I)}
\leq 1$ for maximization problems. In the following we will recover a famous
(randomized) $0.878$-approximation algorithm ([Goemans and Williamson,
1995](#references)) for the maximum cut problem that is based on semidefinite
programming!

First, we need to introduce the graph Laplacian. For an undirected graph $G =
(V, E)$ with $V = \{1, \ldots, n\}$ and edge weights $w_e$, this is the
symmetric matrix $L \in \mathbb{S}^n$ with

$$
    L_{i,j} = \begin{cases}
        \sum_{k = 1}^n w_{\{i, k\}}, &\text{if}~i = j, \\
        -w_{\{i,j\}}, &\text{if}~i \neq j,
    \end{cases}
$$

where we assume $w_{\{i,j\}} = 0$ for $i, j \in V$ with $\{i,j\} \not\in E$.
Informally, $L$ has the weighted vertex degrees on the diagonal and the
negative edge weights on the off-diagonals (whose indices $i, j$ correspond to
edges $\{i, j\}$).

Equipped with the Laplacian matrix $L$, the maximum cut problem can be stated as
follows:

$$
\begin{align*}
    \text{maximize} ~&~ \frac{1}{4} x^T L x \\
    \text{where}
    ~&~ x \in \{-1, 1\}^n \\
\end{align*}
$$

In this form, $x_i = 1$ represents $i \in C$ and $x_i = -1$ represents $x_i \in
V \setminus C$.

---

<div class="alert alert-info">

   **Bonus task 2.1:** Show that, indeed, $w(C) = \frac{1}{4} x^T L x$ for $x$
   defined as above.

   <details>
   <summary style='display: list-item'>Hint</summary>

   - Write $L$ as $L = D + H$ where $D$ is a diagonal matrix and where $H$ is a
     "hollow" matrix with zeros on the diagonal.
   - Compute $x^T D x$ and $x^T H x$, then compare their sum with $w(C)$

   </details>
</div>


---

<div class="alert alert-info">

   **Task 2.2:** Convince your group partner(s) of the following statements
   concerning the matrix $X = xx^T$:

   1. $X$ is symmetric,
   2. $\operatorname{rank}(X) = 1$,
   3. $\operatorname{diag}(X) = \mathbf{1}$, and
   4. $X$ is positive semidefinite, written $X \succeq 0$.

</div>

One can show also the following: If an $n \times n$ matrix $X$ fulfills
properties (1) to (3), then it can be written as $X = xx^T$ with $x \in \{-1,
1\}^n$. Indeed, property (4) already follows from (1) and (2), so there is some
overlap in the above statements.

---

<div class="alert alert-info">

   **Task 2.3:** Rewrite the solution to Task 2.1 to use the matrix $X$ instead
   of $x$.

   <details>
   <summary style='display: list-item'>Recommended hint</summary>

   - Since $w(C)$ is a scalar, $w(C) = \operatorname{Tr}(w(C))$, where
     $\operatorname{Tr}$ denotes the [trace of a square
     matrix](https://en.wikipedia.org/wiki/Trace_(linear_algebra)).
   - Use suited properties of the trace.

   </details>

   <details>
   <summary style='display: list-item'>Notation options</summary>

   - It is $\langle A, B \rangle = \operatorname{Tr}(A^T B) =
     \operatorname{Tr}(A B^T)$ where $\langle \cdot, \cdot \rangle$ denotes the
     [Frobenius inner
     product](https://en.wikipedia.org/wiki/Frobenius_inner_product) for real
     matrices. Both the inner product and the trace notation are common in the
     literature on semidefinite programming.

   </details>
</div>

$$
    w(C) = w(X) = {\color{red}{\text{[some function of $L$ and $X$]}}}
$$

Now, we have rewritten the convex objective $w(C)$ as a linear objective.
(Recall that we are *maximizing*, so a convex objective is not helpful!) Of
course, this does not come for free, as we have implicitly added the non-convex
constraint $X = xx^T$, which in turn is equivalent to the four properties
outlined in Task 2.2. Let's write down what we have so far:

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    \text{maximize} ~&~ w(X) \tag{1} \\
    \text{subject to}
    ~&~ \operatorname{rank}(X) = 1, \tag{1a} \\
    ~&~ \operatorname{diag}(X) = \mathbf{1}, \tag{1b} \\
    ~&~ X \succeq 0, \tag{1c} \\
    \text{where}
    ~&~ X \in \mathbf{S}^n. \tag{1d} \\
\end{align*}
$$

We observe that the objective and constraint (1b) are linear while constraint
(1c) (together with (1d)) enforces membership in the cone of symmetric positive
semidefinite matrices ($S^n_+$); so this part of the problem is both convex and
computationally tractable. The hard part is constraint (1a). If we *just leave
it out*, we obtain a relaxation of the original problem:

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    \text{maximize} ~&~ w(X) \tag{2} \\
    \text{subject to}
    ~&~ \operatorname{diag}(X) = \mathbf{1}, \\
    ~&~ X \succeq 0, \\
    \text{where}
    ~&~ X \in \mathbf{S}^n.\\
\end{align*}
$$


---

<div class="alert alert-info">

   **Task 2.4:** Discuss the following in your group:

   1. Will the optimum objective value of problem (2) be greater or lower than
      that of problem (1)?
   2. Can you explain (or sketch) why the constraint $X \succeq 0$ may be a
      useful addition to problem (2) even though we argued earlier that it is
      redundant for problem (1)?

</div>

---

<div class="alert alert-warning">

   **Take a pause.** We'll discuss solutions before we move on.

</div>

---

<div class="alert alert-info">

   **Task 2.5:** Complete the code below to solve problem (2).

</div>

You'll need some of the following:

| on paper | in picos |
| --- | --- |
| $X \in \mathcal{S}^n$ | `X = pc.SymmetricVariable("X", n)` |
| $A \preceq B$ | `A << B` |
| $A \succeq B$ | `A >> B` |
| $\langle A, B \rangle$ | `(A\|B)` |
| $\operatorname{Tr}(A)$ | `A.tr` |
| $\operatorname{diag}(A)$ | `A.maindiag` |

Here $\operatorname{Tr}(A) = \sum_i A_{ii}$ denotes the trace and
$\operatorname{diag}(A)$ the main diagonal (as a column vector) of $A$.

In [ ]:
import picos as pc

# Load the graph Laplacian of G.
L = pc.Constant("L", nx.laplacian_matrix(G))

# TODO: Implement problem (2).
# X = 
# P = 

# TODO: Solve it. (Reduce the number of vertices if this takes too long.)

# Show solution details.
objective_value = P.value
total_weight = L.tr.value / 2
fraction = objective_value/total_weight
print(
    f"Objective value:   {objective_value:6.3f}\n"
    f"Total edge weight: {total_weight:6.3f}\n"
    f"Fraction:          {fraction:6.3f}"
)

If the above shows an objective value and a fraction of around $0.75$ to $0.8$,
then we have solved the relaxation! But what can we do with the solution? After
all, we would like to make a binary decision for each of the $n$ vertices but
instead we got a continuous $n \times n$ matrix.

If that matrix happens to be of rank one, then we would have solved problem (1).
Let's check:

In [ ]:
rank_X = np.linalg.matrix_rank(X.np, tol=1e-5)
print(f"rank(X) = {rank_X} (n = {n})")

Most likely: no luck! So how does the algorithm of Goemans and Williamson
proceed from here?

First, it uses a [Cholesky
decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition): Every
symmetric positive semidefinite matrix can be written as an outer product of a
lower-triangular matrix with itself, that is

$$
    X = TT^T.
$$

Had we solved problem (1), then it would be $t_i t_j^T = X_{ij} = x_i x_j$ for
all $i$ and $j$, where $t_k$ is the $k$-th **row** of $T$. This would tell us
how to partition the nodes optimally: exactly the vertices with $t_i t_j^T = 1$
go into the same partition. But as we solved the relaxed problem (2), $t_i t_j^T
\in [-1, 1]$ is not necessarily integral and only provides a hint as to whether
vertices $i$ and $j$ should be in the same partition ($t_i t_j^T$ close to $1$)
or not ($t_i t_j^T$ close to $-1$). So let us try to "round" the continuous row
vector $t_i$ to some $x_i \in \{-1, 1\}$.


---

<div class="alert alert-info">

   **Bonus task 2.6:** Discuss the following:

   1. What is $\lVert t_i \rVert$?
   2. Given a random column vector $v \in \mathbb{R}^n$ with $\lVert v \rVert =
      1$, what would be a natural way to map $t_i$ to $1$ or $-1$? (Make a sketch
      in 2D.)

   <details>
   <summary style='display: list-item'>Hint</summary>

   - Use $\operatorname{diag}(X) = \mathbf{1}$.

   </details>

   <details>
   <summary style='display: list-item'>Solution</summary>

   - The algorithm uses $v$ to put $i \in C$ if and only if $t_i v \geq 1$. In
     other words, $x_i = \operatorname{sign}(t_i v)$ and $x =
     \operatorname{sign}(T v)$.

   </details>
</div>

---

<div class="alert alert-info">

   **Task 2.7:** Complete the code below to obtain a partitioning $x \in \{-1,
   1\}^n$.

<details>
<summary style='display: list-item'>Documentation for NumPy functions</summary>

You can use

- [np.linalg.cholesky](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cholesky.html),
- [np.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html),
- [np.random.normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html),
- [np.sign](https://numpy.org/doc/stable/reference/generated/numpy.sign.html).

Recall that NumPy uses `@` for matrix-matrix and matrix-vector multiplication.

</details>

<details>
<summary style='display: list-item'>Hint on generating unit norm vectors uniformly at random</summary>

- The easiest way to pick a point on a $(d-1)$-dimensional unit sphere (embedded
  in $\mathbb{R}^d$) uniformly at random is to sample the $d$-dimensional
  standard normal distribution and normalize the resulting vector.
- (Here, normalization turns out to be unnecessary but doesn't hurt either.)

</details>
</div>

In [ ]:
def analyze_and_plot_partition(G, x):
    assert len(x) == n
    assert np.allclose(np.abs(x), 1)

    inC = vert_data(G, x > 0)
    VmC = vert_data(G, x < 0)
    cut = edge_data(G, x, True)
    unc = edge_data(G, x, False)

    traces = [
        go.Scatter(mode="lines", **cut, line_color="purple", name="cut"),
        go.Scatter(mode="lines", **unc, line_color="lightgray", name="uncut"),
        go.Scatter(mode="markers", **inC, marker_color="red", name="C"),
        go.Scatter(mode="markers", **VmC, marker_color="blue", name="V \\ C"),
    ]

    go.Figure(traces, LAYOUT).show()

    wC = (x.T * L * x / 4).value
    wE = L.tr.value / 2

    print(
        f"n:             {n:6d}\n"
        f"|C|:           {sum(x > 0):6d}\n"
        f"|V \\ C|:      {sum(x < 0):7d}\n"
        f"w(C):          {wC:6.3f}\n"
        f"w(E):          {wE:6.3f}\n"
        f"w(C) / w(E):   {wC/wE:6.3f}\n"
        f"{'-'*21}\n"
        f"Relaxation:    {fraction:6.3f}\n"
        f"Approx. ratio: {(wC/wE)/fraction:6.3f}"
    )

# TODO: Perform a Cholesky decomposition T of X.
# T = 

# TODO: Generate a vector v with unit norm uniformly at random.
# v = 

# TODO: Generate an n-vector x with entries -1 and 1 from T and v.
# x = 

# Show the partition.
analyze_and_plot_partition(G, x)

The expected approximation ratio is at least $0.878$. If all went well, your
program should exceed this at least half of the time. (Rerun the last cell to
get a new random rounding and the whole notebook for a new instance.)

In practice, one could repeat the random projection until the bound is exceeded,
which leads to guaranteed performance but a randomized runtime with at most two
rounds in expectation.

---

<div class="alert alert-info">

   **Bonus task 2.8:** Prove that the expected approximation ratio
   $\frac{\mathbb{E}(w(C))}{\operatorname{OPT}}$ is at least $0.878$ by filling
   in the gaps. You will need the following:

   1. Another relaxation of MAXCUT (closely related to the Cholesky
      decomposition argument) is the following program:

      $$
        \begin{align*}
            \text{maximize} ~&~
              \sum_{\{i, j\} \in E} w_{\{i,j\}}
              \frac{1 - \langle t_i, t_j \rangle}{2} \tag{3} \\
            \text{subject to}
            ~&~ \lVert t_i \rVert = 1 \quad \text{for}~i \in \{1, \ldots, n\} \\
            \text{where}
            ~&~ t_i \in \mathbb{R}^n \quad \text{for}~i \in \{1, \ldots, n\} \\
        \end{align*}
      $$

      (To see that this is a relaxation, take $t_i = x_i e_1$ for all $i$.) We
      can denote its optimum value as $\operatorname{REL}$. Since this is a
      relaxation, $\operatorname{OPT} \leq \operatorname{REL}$.
   2. The angle between two unit vectors $t_i$ and $t_j$ in radians is
      $\arccos{\langle t_i, t_j \rangle}$. Use this to describe the probability
      $\mathbb{P}(x_i x_j = 1)$. (Make use of your sketch from task 2.6!)
   3. It is $$\frac{\arccos{\tau}}{\pi} \geq 0.878 \frac{1 - \tau}{2}$$ for all
      $\tau \in [-1, 1]$.

</div>

<div hidden>

$$
\providecommand{\TODO}{}\renewcommand{\TODO}{{\color{red}{[\ldots]}}}
$$

</div>

$$
\begin{align*}
    \mathbb{E}(w(C))
    &= \sum_{\{i, j\} \in E} w_{\{i, j\}} \mathbb{P}(x_i x_j = 1) \\
    &= \sum_{\{i, j\} \in E} w_{\{i, j\}} \TODO \\
    &\geq 0.878 \left( \sum_{\{i, j\} \in E} w_{\{i, j\}} \TODO \right) \\
    &= 0.878 \cdot \TODO \\
    &\geq 0.878 \cdot \operatorname{OPT}.
\end{align*}
$$

### Additional resources

- A quantum analog of the maximum cut problem is studied in a recent preprint by
  King ([2022](#references)) and in earlier work, e.g. by Gharibian and Parekh
  ([2019](#references)).

## References

- Gharibian and Parekh. 2019. Almost optimal classical approximation algorithms
  for a quantum generalization of Max-Cut. arXiv preprint arXiv:1909.08846.
  https://doi.org/10.48550/arXiv.1909.08846
- Michel X. Goemans and David P. Williamson. 1995. Improved approximation
  algorithms for maximum cut and satisfiability problems using semidefinite
  programming. *J. ACM* 42, 6 (Nov. 1995), 1115–1145.
  https://doi.org/10.1145/227683.227684
- Robbie King. 2022. An Improved Approximation Algorithm for Quantum Max-Cut.
  arXiv preprint arXiv:2209.02589. https://doi.org/10.48550/arXiv.2209.02589